# Numerical Benchmarks

In [2]:
import sys
sys.path.insert(0, '../../../src_tf/')

import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
import multiprocessing as mp
import random
import pickle

from qiskit.quantum_info import DensityMatrix, random_unitary
from qiskit.quantum_info import Operator
from scipy.linalg import sqrtm
from tqdm.notebook import tqdm

from loss_functions import *
from optimization import *
from quantum_channel import *
from kraus_channels import *
from lindblad_channels import *
from quantum_tools import *
from experimental import *
from spam import *

#np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(precision=4)

#import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

## Recover SPAM and Map

### Generate True Model, Full POVM and Inital

In [11]:
def generate_spam_benchmark(n=3, c1=1, c2=1):
    d = 2**n

    init_target = InitialState(d, c=c1)
    povm_target = POVM(d, c=c2)

    spam_target = SPAM(init = init_target,
                       povm = povm_target)
    
    return spam_target


def generate_spam_data(spam_target, N_spam=None, shots=1024):
    n = int(np.log2(spam_target.d))
    inputs_spam, _ = generate_pauliInput_circuits(n)
    N_spam = inputs_spam.shape[0]

    state = tf.repeat(spam_target.init.init[None,:,:], N_spam, axis=0)
    targets_spam = measurement(state, U_basis = inputs_spam, povm = spam_target.povm.povm)

    #add noise
    targets_spam = add_shot_noise(targets_spam, shots = shots)
    return inputs_spam, targets_spam


def generate_map_data(channel_target, spam_target, N_map=None, shots=1024):
    n = int(np.log2(channel_target.d))
    inputs_map, _ = generate_pauli_circuits(n = n, 
                                            circuit_target=None,  
                                            trace=False,
                                            N=N_map)
    U_prep, U_basis = inputs_map

    N_map = U_prep.shape[0]
    state = tf.repeat(tf.expand_dims(spam_target.init.init, axis=0), N_map, axis=0)
    state = apply_unitary(state, U_prep)
    state = channel_target.apply_channel(state)
    targets_map = measurement(state, U_basis, spam_target.povm.povm)
    
    #add noise
    targets_map = add_shot_noise(targets_map, shots = shots)

    return inputs_map, targets_map


def model_pipeline(channel_target = None, shots=1024, N_map = None, num_iter_spam=None, num_iter_map=None, verbose=False):
    # Make Benchmark
    #################################################################################
    n = 3
    d = 2**n
    
    spam_target = generate_spam_benchmark(n=n, c1=0.9, c2=0.9)
    inputs_spam, targets_spam = generate_spam_data(spam_target, N_spam=None, shots=shots)

    inputs_map, targets_map = generate_map_data(channel_target, spam_target, N_map=N_map, shots=shots)
    #################################################################################

    # Fit Models
    #################################################################################
    spam_model = SPAM(init = InitialState(d, c=0.9),
                      povm = CorruptionMatrix(d, c=0.9),
                      optimizer = tf.optimizers.Adam(learning_rate=0.01))
    
    #spam_model.pretrain(300, verbose=False)

    spam_model.train(inputs = inputs_spam,
                     targets = targets_spam,
                     num_iter = num_iter_map,
                     verbose = False,
                    )

    model = ModelQuantumMap(channel = KrausMap(d = d, 
                                               rank = d**2,
                                               spam = spam_model,
                                              ),
                            loss_function = ProbabilityMSE(),
                            optimizer = tf.optimizers.Adam(learning_rate=0.01),
                            logger = Logger(loss_function = ProbabilityMSE(),
                                            loss_function_val = channel_fidelity_loss,
                                            sample_freq = 400),
                           )

    model.train(inputs = inputs_map,
                targets = targets_map,
                inputs_val = None,
                targets_val = [channel_target],
                num_iter = num_iter_map,
                N = 500,
                verbose = verbose,
                )
    #################################################################################
    model.optimizer = None
    spam_model.optimizer = None
    model.inputs = None
    model.targets = None
    model.inputs_val = None
    model.targets_val = None

    return model

## Fit Models

In [12]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

n = 3
d = 2**n

channel_target1 = CompactLindbladMap(d, 1, 1, 0.1)
model1 = model_pipeline(channel_target1, 
                        shots=1024, 
                        N_map=2000-6**n, 
                        num_iter_spam = 4000,
                        num_iter_map = 4000,
                        verbose=True)

channel_target2 = CompactLindbladMap(d, 16, 10000, 0.001)
model2 = model_pipeline(channel_target2, 
                        shots=1024, 
                        N_map=2000-6**n, 
                        num_iter_spam = 4000,
                        num_iter_map = 4000,
                        verbose=True)

channel_target3 = CompactLindbladMap(d, 16, 100, 0.001)
model3 = model_pipeline(channel_target3, 
                        shots=1024, 
                        N_map=2000-6**n, 
                        num_iter_spam = 4000,
                        num_iter_map = 4000,
                        verbose=True)

channel_target4 = CompactLindbladMap(d, 8, 1, 0.01)
model4 = model_pipeline(channel_target4, 
                        shots=1024, 
                        N_map=2000-6**n, 
                        num_iter_spam = 4000,
                        num_iter_map = 4000,
                        verbose=True)


saver([[channel_target1, channel_target2, channel_target3, channel_target4],
       [model1, model2, model3, model4]], "data/"+"atypical_maps_benchmark.model")

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

0.24358633882144867 -0.09805713185539307
0.007367782150606879 -0.8642702030215942
0.006394330220305683 -0.9237237307125933
0.0060414664590011805 -0.934768560271813
0.006409520378915215 -0.9381282050593764
0.00629745596857743 -0.9390359839824791
0.006121570488609958 -0.9394302456412726
0.006331951074818019 -0.9395891701521448
0.006094469512211761 -0.9399195976198793
0.0062606237621670915 -0.9396955742979446
0.006297382033488429 -0.9393658964677797


  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

...

  0%|          | 0/4000 [00:00<?, ?it/s]

0.4922248695655371 -0.17476615021428712
0.007440001487497792 -0.8460975367234452
0.006033645249702766 -0.8933560745457293
0.0059721782231863954 -0.9048559197020969
0.006042411031887418 -0.9067045822849354
0.005594167468542802 -0.9056601695490609
0.0057358340091380675 -0.903499023132544
0.005681883451294178 -0.9014280102363719
0.005902302648657004 -0.899888946891387
0.005686281789427228 -0.898740394927612
0.005785784370853339 -0.8982353045433719


  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

0.11688506897008048 -0.4470260988959273
0.006359642207139404 -0.864111823673686
0.00553955582537517 -0.8791847382457072
0.005455525626897918 -0.8701043598770587
0.005087218628523157 -0.8601257999916526
0.005289084545669357 -0.8515079330667749
0.0054650098888782755 -0.845793798263548
0.005337782935298238 -0.8414387901593353
0.005345751146038675 -0.8386827236952399
0.005200871616562653 -0.8358876396026531
0.005104225960533267 -0.834191775016569


## Sample and Noise Sensitivity

In [14]:
n = 3
d = 2**n
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

channel_target = CompactLindbladMap(d, 1, 1, 0.1)
model_ns1 = model_pipeline(channel_target, 
                           shots=256, 
                           N_map=256, 
                           num_iter_spam = 4000,
                           num_iter_map = 4000,
                           verbose=True)


model_ns2 = model_pipeline(channel_target, 
                           shots=1024, 
                           N_map=1024, 
                           num_iter_spam = 4000,
                           num_iter_map = 4000,
                           verbose=True)

model_ns3 = model_pipeline(channel_target, 
                           shots=4096, 
                           N_map=4096, 
                           num_iter_spam = 4000,
                           num_iter_map = 4000,
                           verbose=True)

saver([[channel_target],
       [model_ns1, model_ns2, model_ns3]], "data/"+"atypical_maps_robustness.model")

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

0.25255740329077125 -0.09790789532063889
0.016885521111175632 -0.7222070780249391
0.015452065602110869 -0.760714612585643
0.01531860940478661 -0.7641966034332381
0.015297122467690733 -0.7642737530500818
0.015292475955475414 -0.7639739577406761
0.015291226336929678 -0.7637341865100247
0.015290815872548092 -0.7635822063635365
0.015290654856823335 -0.7634848393048708
0.01529058398287721 -0.7634169420276298
0.015290549716756884 -0.7633666378726003


  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

0.27708061330793593 -0.09534488348365604
0.007335178428198955 -0.848321899569145
0.006000436057100031 -0.9126782156148544
0.006099827470396115 -0.9254013088917373
0.005813236066433264 -0.9287196248575446
0.005922847688239885 -0.9301088602481353
0.005848523504409336 -0.9302654758922971
0.005760523656058943 -0.9307514814209986
0.005870826412870416 -0.9307074604315804
0.005738973434337544 -0.9305317511651956
0.005925678601175935 -0.9304364126511478


  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

0.23703334880839866 -0.09578037331174998
0.0029864486414783366 -0.8720496424790909
0.0019357776372289142 -0.9424086891684207
0.0018231443703856038 -0.9612835439292674
0.0017617372507687896 -0.9683585702097184
0.0017684640141524685 -0.9713771371291521
0.0017746410494197695 -0.972789048951825
0.001789841067956938 -0.9735358872083617
0.0017617855700840378 -0.9736753982369741
0.0016946056043024946 -0.973672610133213
0.0017370244957168452 -0.9738120782717761
